In [1]:
words =  open("names.txt", "r").read().splitlines()
words[0]

'emma'

In [2]:
import torch
import torch.nn.functional as F

In [3]:
xin = []
yin = []
for word in words:
    chs = ['.'] + list(word) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        # print(ch1, ch2, ch3)
        xin.append(ch1)
        xin.append(ch2)
        yin.append(ch3)
        # if ch3 == '.':
        #     break
# print(words[:3])
# print(xin[:20])
# print(yin[:10])


chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}

def stoi2(s, t):
    return stoi[s] * len(stoi) + stoi[t]

def itos2(i):
    return itos[i // len(stoi)] + itos[i % len(stoi)]

# print(stoi['m'])
# print(stoi2('e', 'e'))
# print(itos2(15))
# print(itos[13])

xs, ys = [], []

for ch1, ch2, ch3 in zip(xin[::2], xin[1::2], yin):
    xs.append(stoi2(ch1, ch2))
    ys.append(stoi[ch3])

xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [294]:
h_l = 200
emb_space = 20

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((729, emb_space), generator=g, requires_grad=True)

W1 = torch.randn((C.shape[1], h_l), generator=g, requires_grad=True)
b1 = torch.randn((h_l), generator=g, requires_grad=True)

W2 = torch.randn((h_l, 27), generator=g, requires_grad=True)
b2 = torch.randn((27), generator=g, requires_grad=True)

parameters = [C, W1, b1, W2, b2]

In [83]:
C.shape

torch.Size([729, 50])

In [325]:
#forward pass
for _ in range(10000):
    # mini_batch
    batch = torch.randint(0, xs.shape[0], (32,)) 
    emb = C[xs[batch]]
    h = torch.tanh(emb@W1 + b1)
    logits = h@W2 + b2
    loss = F.cross_entropy(logits, ys[batch])
    #backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    for p in parameters:
        p.data += -0.0001 * p.grad
print(loss.item())

1.8815360069274902


In [326]:
emb = (C[xs]).view(-1, C.shape[1])
h = torch.tanh(emb@W1 + b1)
logits = h@W2 + b2
loss = F.cross_entropy(logits, ys)
print(loss.item())

2.0787549018859863


In [327]:
def get_probability_distribution(input_chars):
    # Convert input characters to index
    ix = stoi2(input_chars[0], input_chars[1])
    
    # Get embedding for the input
    emb = C[ix].unsqueeze(0)
    
    # Calculate hidden layer
    h = torch.tanh(emb @ W1 + b1)
    
    # Calculate logits
    logits = h @ W2 + b2
    
    # Apply softmax to get probabilities
    probs = F.softmax(logits, dim=1)
    
    # Convert to numpy for easier handling
    probs_np = probs.detach().numpy()[0]
    
    # Create a dictionary of character probabilities
    char_probs = {itos[i]: prob for i, prob in enumerate(probs_np)}
    
    return char_probs

# Example usage
input_chars = 'ia'  # Example input
prob_dist = get_probability_distribution(input_chars)

# Print probabilities sorted by value
for char, prob in sorted(prob_dist.items(), key=lambda x: x[1], reverse=True):
    print(f"{char}: {prob:.4f}")

.: 0.3696
n: 0.3201
h: 0.1915
s: 0.0229
r: 0.0207
m: 0.0200
l: 0.0094
a: 0.0070
g: 0.0062
v: 0.0046
y: 0.0041
d: 0.0039
t: 0.0031
i: 0.0029
b: 0.0025
u: 0.0024
k: 0.0017
e: 0.0016
p: 0.0013
q: 0.0013
o: 0.0009
c: 0.0008
z: 0.0008
w: 0.0004
f: 0.0000
j: 0.0000
x: 0.0000


In [328]:
def generate_names(num_names):
    g = torch.Generator().manual_seed(2147483648)
    names = []
    
    for _ in range(num_names):
        name = []
        ch1, ch2 = '.', '.'  # Start with '..' as the initial bigram
        
        while True:
            prob_dist = get_probability_distribution(ch1 + ch2)
            probs = torch.tensor(list(prob_dist.values()))
            next_char_idx = torch.multinomial(probs, num_samples=1, replacement=True, generator=g).item()
            next_char = itos[next_char_idx]
            
            if next_char == '.':  # End of name
                break
            
            name.append(next_char)
            ch1, ch2 = ch2, next_char
        
        names.append(''.join(name))
    
    return names

# Generate and print 5 names
generated_names = generate_names(15)
for i, name in enumerate(generated_names, 1):
    print(f"Name {i}: {name}")

Name 1: aan
Name 2: ainor
Name 3: alea
Name 4: em
Name 5: emareiagdalah
Name 6: aphali
Name 7: anah
Name 8: avinsham
Name 9: eyah
Name 10: eni
Name 11: olla
Name 12: larydne
Name 13: aubris
Name 14: eilyna
Name 15: eahikenainexx
